# Querying Files & Creating Tables (From Raw Files to Delta Lake)
## 1. Introduction

This notebook will guide you through:
- Querying raw files directly
- Registering external tables on data sources
- Creating Delta tables using CTAS
- Using the new `read_files()` function
- Understanding limitations of external vs Delta tables
- Hands-on exercises

We’ll work with sample bookstore data:
- Customers (JSON)
- Books (CSV)

In [0]:
%python
import os
import json

# Customer JSON file contents
customer_files = {
    "export_001.json": [
        {"id": 1, "name": "Alice", "email": "alice@example.com"},
        {"id": 2, "name": "Bob", "email": "bob@example.com"}
    ],
    "export_002.json": [
        {"id": 3, "name": "Charlie", "email": "charlie@example.com"},
        {"id": 4, "name": "Diana", "email": "diana@example.com"}
    ],
    "export_003.json": [
        {"id": 5, "name": "Eve", "email": "eve@example.com"},
        {"id": 6, "name": "Frank", "email": "frank@example.com"}
    ],
    "export_004.json": [
        {"id": 7, "name": "Grace", "email": "grace@example.com"},
        {"id": 8, "name": "Henry", "email": "henry@example.com"}
    ],
    "export_005.json": [
        {"id": 9, "name": "Ivy", "email": "ivy@example.com"},
        {"id": 10, "name": "Jack", "email": "jack@example.com"}
    ]
}

# Book CSV file contents
book_files = {
    "export_001.csv": "id,title,author\n1,Book A,Author X\n2,Book B,Author Y",
    "export_002.csv": "id,title,author\n3,Book C,Author Z\n4,Book D,Author W",
    "export_003.csv": "id,title,author\n5,Book E,Author Q\n6,Book F,Author P",
    "export_004.csv": "id,title,author\n7,Book G,Author L\n8,Book H,Author M"
}

# Folder in DBFS where files will be saved
base_path_customers = "/dbfs/mnt/tutorial/customers-json/"
base_path_books = "/dbfs/mnt/tutorial/books-csv/"

# Create directories
os.makedirs(base_path_customers, exist_ok=True)
os.makedirs(base_path_books, exist_ok=True)

# Write customer JSON files
for filename, content in customer_files.items():
    with open(os.path.join(base_path_customers, filename), "w") as f:
        for record in content:
            f.write(json.dumps(record) + "\n")

# Write book CSV files
for filename, content in book_files.items():
    with open(os.path.join(base_path_books, filename), "w") as f:
        f.write(content)

print("Customer JSON + Book CSV files uploaded to /mnt/tutorial/")

In [0]:
%python
display(dbutils.fs.ls("/mnt/tutorial/customers-json/"))
display(dbutils.fs.ls("/mnt/tutorial/books-csv/"))

## 2. Querying Data Files Directly

Databricks allows you to query files directly with SQL without creating a table.

### 2.1 Self-describing formats

Self-describing formats (e.g., JSON, Parquet, Avro) contain schema information.

In [0]:
%python
# Set a Spark configuration value
spark.conf.set("dataset.bookstore", "/mnt/tutorial")

# Get it back in Python
print(spark.conf.get("dataset.bookstore"))

In [0]:
-- Query a single JSON file
SELECT * FROM json.`${dataset.bookstore}/customers-json/export_001.json`

In [0]:
-- Query multiple JSON files with a wildcard
SELECT * 
FROM json.`${dataset.bookstore}/customers-json/export_*.json`;

In [0]:
-- Query the entire directory
SELECT * 
FROM json.`${dataset.bookstore}/customers-json`;
-- Use case: Quickly validate or explore file contents before creating tables.

In [0]:
-- Count number of customers
SELECT count(*) 
FROM json.`${dataset.bookstore}/customers-json`;

### 2.2 Non-self-describing formats

For text, CSV, or binary files, Databricks doesn’t know the schema automatically.

In [0]:
%sql
-- Read as raw text (unparsed)
SELECT * FROM text.`${dataset.bookstore}/customers-json`;

In [0]:
-- Read as raw binary (file bytes)
SELECT * FROM binaryFile.`${dataset.bookstore}/customers-json`;

In [0]:
-- Read raw CSV (not parsed, no schema)
SELECT * FROM csv.`${dataset.bookstore}/books-csv`;
-- Here, schema is missing → need to manually define it when registering as a table.

### 2.3 Metadata: file source

In [0]:
-- Get the source file name
SELECT *, input_file_name() AS source_file
FROM json.`${dataset.bookstore}/customers-json`;
-- Use case: Debugging or tracing records back to their source file.

## 3. Registering Tables on External Data Sources

Instead of querying files every time, you can register external tables that point to file locations.

### 3.1 Example: External CSV table

In [0]:
USE hive_metastore.default;

CREATE TABLE books_csv
(book_id STRING, title STRING, author STRING)
USING CSV
OPTIONS (
  header = "true",
  delimiter = ";"
)
LOCATION "${dataset.bookstore}/books-csv";

DESCRIBE EXTENDED books_csv;

-- Shows that it’s an external table with provider = CSV , non delta table

In [0]:
SELECT * FROM books_csv;

### 3.2 Adding more files

In [0]:
%python
dataset_bookstore = "dbfs:/mnt/tutorial"  # Set base path

(spark.read
 .table("books_csv")  # Read Hive/Delta table
 .write
 .mode("append")      # Append mode
 .format("csv")       # Save as CSV
 .option("header", "true")
 .option("delimiter", ";")
 .save(f"{dataset_bookstore}/books-csv"))  # Save to DBFS

In [0]:
%python
## Check new files: 
files = dbutils.fs.ls(f"{dataset_bookstore}/books-csv")
display(files)

In [0]:
-- But Note:
SELECT count(*) FROM books_csv;
-- Cached, doesn’t pick new files automatically


In [0]:
-- Solution: Refresh table
REFRESH TABLE books_csv;

In [0]:
SELECT count(*) FROM books_csv;

#### Limitations of external non-Delta tables
- Not Delta → no Time Travel
- Limited schema evolution support
- Performance issues on large datasets (no transaction log)
- Schema must be defined manually

## 4. Creating Delta Tables with CTAS

- Delta Lake is the default format in Databricks.
- **Best practice:** Use CTAS (Create Table As Select) to convert external sources → Delta tables.

### 4.1 JSON → Delta

In [0]:
CREATE TABLE customers AS
SELECT * 
FROM json.`${dataset.bookstore}/customers-json`;

DESCRIBE EXTENDED customers;
-- Now customers is a Delta managed table. Schema was automatically inferred.

### 4.2 CSV → Delta (unparsed vs parsed)

In [0]:
-- Unparsed CSV:
CREATE TABLE books_unparsed AS
SELECT * 
FROM csv.`${dataset.bookstore}/books-csv`;

In [0]:
select * from books_unparsed;

In [0]:
-- Solution: Using temp view with schema:
DROP VIEW IF EXISTS books_tmp_vw;
CREATE TEMP VIEW books_tmp_vw
(book_id STRING, title STRING, author STRING, category STRING, price DOUBLE)
USING csv
OPTIONS (
  path = "${dataset.bookstore}/books-csv/export_*.csv",
  header = "true",
  delimiter = ";"
);


In [0]:
CREATE TABLE books AS
SELECT * FROM books_tmp_vw;
-- Now books is a Delta table with schema inferred.

DESCRIBE EXTENDED books;

## 5. Advanced: Querying with read_files()

Databricks recently introduced read_files() for easier direct querying.

### 5.1 Example: CSV query
- Automatically infers a unified schema across all files.
- Stores mismatched values in _rescued_data column.

In [0]:
SELECT * FROM read_files(
  '${dataset.bookstore}/books-csv/export_*.csv',
  format => 'csv',
  header => 'true',
  delimiter => ';'
);

### 5.2 Create Delta table directly from read_files

In [0]:
DROP TABLE IF EXISTS books;
CREATE TABLE books
AS SELECT * FROM read_files(
    '${dataset.bookstore}/books-csv/export_*.csv',
    format => 'csv',
    header => 'true',
    delimiter => ','
);

DESCRIBE EXTENDED books;
-- Delta table created directly.
-- Schema inference handled automatically.

## The _metadata Column
The input_file_name() function is no longer supported in newer versions of the Databricks Runtime. As an alternative, you can use the _metadata.file_path attribute to retrieve the file path information.

In [0]:
SELECT *,
       _metadata.file_path AS source_file
FROM json.`${dataset.bookstore}/customers-json`;

In [0]:
SELECT *,
       _metadata.file_name AS source_file
FROM json.`${dataset.bookstore}/customers-json`;

In [0]:
SELECT *,
       _metadata.file_size AS source_file
FROM json.`${dataset.bookstore}/customers-json`;

In [0]:
SELECT *,
       _metadata.file_modification_time AS source_file
FROM json.`${dataset.bookstore}/customers-json`;

By leveraging the _metadata column, you can access various details about your input files, such as:

**_metadata.file_path:** The full path to the input file.

**_metadata.file_name:** The name of the file, including its extension.

**_metadata.file_size:** The size of the file in bytes.

**_metadata.file_modification_time:** The timestamp of the last modification made to the file.

## 6. Summary

- Query files directly → Quick exploration (select * from json.path``).
- External tables → Point to raw files, but limited (not Delta).
- CTAS → Delta tables → Schema inferred automatically, Delta benefits (time travel, ACID, schema evolution).
- read_files() → Simplifies querying multiple files and creating Delta tables directly.
- The _metadata Column